In [ ]:
import pandas as pd

df=pd.read_csv('/home/bram/Project_PD/PD project/Feature matrix/all_notes_annotations/ALLHOSPITALS_NEW_DATA.csv')
print(len(df))
print(df.columns)
print(df['BDSPPatientID'].nunique())

print(len(df))
print(df.columns)
print(df)

import pandas as pd
import re
import nltk
from nltk.stem import SnowballStemmer

# Ensure you have the necessary nltk resources
nltk.download('punkt')

# Initialize the Snowball Stemmer for English
stemmer = SnowballStemmer("english")



keywords = [
    "parkinson disease", "pd", "parkinson", 'parkinsonismx', "disease", "paralysis agitans", "primary parkinsonism", 
    "pdd", "akathisia", "alien limb", "alpha synuclein", "anosmia", "anticholinergic side effect", 
    "apraxia", "autonomic", "axial", "basophilic", "blepharospasm", "bradyphasia", "bradyphonia", 
    "bradyphrenia", "camptocormia", "cholinergic", "cogwheel", "cognitive impairment", "corticobasal", 
    "daws", "denervation", "dopamine", "dopamine agonist", "dysphagia", "dysphonia", "dystonia", 
    "facial", "fasciculation", "gba", "gait", "festinating", "shuffling", "freezing", "globus", 
    "hyposmia", "hyperreflexia", "hypokinetic", "hypomimia", "hypophonia", "idiopathic", "imaging", 
    "dat spect", "datscan", "dat", "mri", "scan", "lrrk2", "snca", "prkn", "pink1", "dj-1", "gba", 
    "vps35", "atp13a2", "lrrk1", "uchl1", "mao-b", "monoamine oxidase b inhibitor", "masked facies", 
    "medullary", "micrographia", "mild cognitive impairment", "mci", "montreal cognitive assessment", 
    "moca", "movement disorder", "multiple system atrophy", "msa", "nigra pars compacta", "nigrostriatal", 
    "ocular torticollis", "occupational therapy", "orthostatic", "orofacial", "pallidotomy", 
    "pallidothalamic", "palliative", "paratonia", "pathology", "pet", "physical therapy", "postencephalitic", 
    "posture", "primary progressive aphasia", "ppa", "prodrome", "progressive supranuclear palsy", 
    "psp", "pyramidal", "rem sleep behavior disorder", "rbd", "restless legs", "saccade", "sialorrhea", 
    "speech therapy", "spasmodic", "stooped", "subthalamic nucleus", "stn", "striatal dopamine deficiency", 
    "stiff", "rigidity", "subcortical", "supranuclear", "swallowing", "tetany", "thalamotomy", "tics", "toronto cognitive assessment", "torca", "tremor", "uptake", "vascular", "nausea", 
    "hallucination", "constipation", "fatigue", "insomnia", "depression", "anxiety", "psychosis", 
    "dosage adjustment", "physiotherapy", "lifestyle changes", "support group", "caregiver", 
    "medication adherence", "prescription refill", "fall precaution", "daily function", "self care", 
    "quality of life", "disease progression", "regular follow-up", "exercise", "sleep hygiene", 
    "bone health", "care coordination", "shuffling gait", "exelon patch", "exelon", "cane", "fall", 
    "deep brain stimulation", "dbs", "bradykinesia", "hypokinesia", "dementia", "pulmonary complication", 
    "pneumonia"
]



In [ ]:
nltk.download('punkt_tab')

In [ ]:

# Function to stem each word in a phrase
def stem_phrase(phrase, stemmer):
    tokens = nltk.word_tokenize(phrase)
    return ' '.join([stemmer.stem(token) for token in tokens])



In [ ]:
# Generate regex patterns
def generate_patterns(keywords):
    neg_patterns = {}
    pos_patterns = {}
    for phrase in keywords:
        stemmed_phrase = stem_phrase(phrase, stemmer)
        words = stemmed_phrase.split()
        neg_pattern = r"(?:(?:\bno\b|\bnot\b|n't|absent|negat)[^,.]{{0,30}}\b{pattern})\b|\b(?:{pattern}\b[^.]{{0,40}}(?:absent|negat))".format(
            pattern= r''.join([f"{word}\s*" for word in words]) #+ r"\b"
        )
        neg_patterns[phrase] = neg_pattern
        pos_patterns[phrase] = r"\b" + r"".join([f"{word}\s*" for word in words]) + r"\b" 
    return neg_patterns, pos_patterns

neg_patterns, pos_patterns = generate_patterns(keywords)

In [ ]:
# Print generated patterns for verification
print("Negated Patterns:")
for key, pattern in neg_patterns.items():
    print(f"{key}: {pattern}")

print("\nPositive Patterns:")
for key, pattern in pos_patterns.items():
    print(f"{key}: {pattern}")

# Function to preprocess text (lowercase, stemming using NLTK)

def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r"\b[a-z]*parkinsonism[a-z]*\b", "parkinsonismx", text)  # Replace words containing 'parkinsonism'
    # print (text)
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token != "'s"]
    return ' '.join(stemmed_tokens)





In [ ]:
def check_patterns(text, neg_patterns, pos_patterns):
    matches = {f"{key}_pos": 0 for key in pos_patterns}
    matches.update({f"{key}_neg": 0 for key in neg_patterns})

    for key in pos_patterns:
        pos_matches = [(m.start(), m.end()) for m in re.finditer(pos_patterns[key], text)]
        neg_matches = [(m.start(), m.end()) for m in re.finditer(neg_patterns[key], text)]

        for start, end in pos_matches:
            if not any(n_start <= start <= n_end or n_start <= end <= n_end for n_start, n_end in neg_matches):
                matches[f"{key}_pos"] = 1

        for start, end in neg_matches:
            matches[f"{key}_neg"] = 1

    return matches

In [ ]:
# Apply preprocessing and pattern checking to each note
df['processed_note'] = df['note_text'].apply(preprocess_text)

# Display the preprocessed notes for verification
print("Preprocessed Notes:")
print(df['processed_note'])

pattern_features = df['processed_note'].apply(lambda note_text: check_patterns(note_text, neg_patterns, pos_patterns))

# Convert the pattern features to a DataFrame
pattern_columns = pd.DataFrame(pattern_features.tolist())

# Concatenate with the original DataFrame
result = pd.concat([df[['BDSPPatientID', 'CreateDate', 'hospital']], pattern_columns], axis=1)

# Display the result
print(result)


In [ ]:
result.to_csv('FEATMATRIX_TEXT.csv', index=False)